# Правила игры
 Используется колода из 52-х карт. Игра рассчитана на 2-4 игроков. В случае 4-х игроки деляться на пары с напротив сидящим и счет ведется для пары.
 
 В начале игры раздающий выдает каждому игроку по 4 карты (в закрытую) по часовой стрелке, начиная со следующего после него, и ещё 4 карты выкладываются на стол (в открытую). Первым ходит игрок следующий за раздающим по часовой стрелке. После каждой партии роль раздающего переходит по часовой стрелке.
 
 Игрок используя одну из карт с руки должен либо составить из числовых карт сумму, равную 11 (в сочетании со скольки угодно картами со стола; туз = 1) и взять эти карты себе в отбой, либо с помощью дамы или короля взять со стола соответсвующую карту, либо забрать все числовые карты и вальтов со стола используя вальта.
 
 При завершении партии игроки начинают подсчет карт в своих отбоях (для пар отбой общий). Очки можно получить если:
 - 1 очко если в отбое игрока есть "♣2"
 - 1 очко если в отбое игрока есть "♢10"
 - 1 очко если кол-во карт масти "♣" больше, чем у остальных (в случае ничьи очко не достается никому)
 - 2 очка если кол-во всех карт в отбое больше, чем у остальных игроков (в случае ничьи оба получают по 1-му очку)


In [ ]:
from random import shuffle as rd_shuffle
#import collections

In [ ]:
class Card(object):
    def __init__(self, name, value, suit, symbol):
        self.value = value  # значение (для вычисления сумм)
        self.suit = suit  # масть
        self.name = name  # название (полное значение в string)
        self.symbol = symbol  # обозначение
        self.showing = False  # для отображения/скрытия
        self.calculatePriority()  # нужно для бота
    
    # назначение карте приоритета (для бота)
    def calculatePriority(self):
        self.priority = 1
        
        if self.suit == 'Clubs':
            # повышаем ценность крестей для бота
            self.priority += 1
            if self.name == 'Two':
                # особый приоритет двойке, чтобы взял при первой возможности
                self.priority += 9
        
        if self.name == 'Jack':
            # понижаем приоритет вальтов, чтобы бот не торопился их использовать
            self.priority -= 6
        
        if self.suit == 'Diamonds' and self.name == 'Ten':
            # аналогично особый приоритет десятке бубей
            self.priority += 9


class Deck(object):
    def shuffle(self, times=1 ): # тасовка карт
        rd_shuffle(self.cards)
    
    def deal(self):  # взятие карты из колоды
        return self.cards.pop(0)

# колода для игры в 11
class Deck11(Deck):
    def __init__(self):
        self.cards = []
        suits = {"Hearts":"♡", "Spades":"♠", "Diamonds":"♢", "Clubs":"♣"}
        values = {"Ace": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5, "Six": 6, "Seven": 7, "Eight": 8, "Nine": 9, "Ten": 10, 
                  "Jack": 'J', "Queen": 'Q', "King": 'K'}
        
        # создание колоды
        for name in values:
            for suit in suits:
                symbolIcon = suits[suit]
                
                if values[name] is int and values[name] != 1: # 1 < values[name] < 11:
                    symbol = symbolIcon + str(values[name])
                else:
                    symbol = symbolIcon + name[0]
                
                self.cards.append( Card(name, values[name], suit, symbol) )

# игрок
class Player(object):
    def __init__(self, name, bot=False):
        self.cards = []   # карты в руке
        self.taken = []   # отбой игрока
        self.name = name  # имя игрока/бота
        self.score = 0    # набранные очки
        self.isBot = bot  # чтобы отличать бота от игрока
    
    def newGame(self):
        self.cards = []
        self.taken = []
        
    def addHandCards(self, cards):  # добавление карт в руку игрока (всегда добавляется по 4 карты)
        for card in cards:
            self.cards.append(card)
    
    def addTakenCards(self, cards): # добавление карт в отбой игрока, в осномном по несколько карт за раз
        for card in cards:
            self.taken.append(card)
    
    def removeHandCard(self, card): # удаление карты с руки
        for i, hc in enumerate(self.cards):
            if hc == card:
                self.cards.pop(i)
                break
    
    def checkCombination(self, fromTable, handCard):
        if handCard.value in ['Q', 'K']:
            if len(fromTable) == 1 and fromTable.value == handCard.value:
                return True
        
        elif handCard.value == 'J' and not any([tCard.value in ['Q', 'K'] for tCard in fromTable]):
            return True
        
        elif len(fromTable) > 0 and all([type(c.value) is int for c in fromTable]):
            if sum([c.value for c in fromTable]) + handCard.value == 11:
                return True
        
        return False
    
    def calculateScore(self): # подсчет карт по окончанию партии
        self.allCardsCount = len(self.taken)
        self.clubsCardsCount = 0
        self.clubsTwo = False
        self.diamondsTen = False
        
        for card in self.taken:
            if card.suit == 'Clubs':
                self.clubsCardsCount += 1
                if card.name == 'Two':
                    self.clubsTwo = True
            
            elif card.suit == 'Diamonds' and card.name == 'Ten':
                self.diamondsTen = True

# бот
class Bot(Player):
    def __init__(self, name):
        Player.__init__(self, name, bot=True)
    
    # подбор наименее нужной карты на руке для сброса
    def chooseLessPriorityCard(self):
        # учесть вальта, нежелательно его выбрасывать  раньше времени 
        minPrior = 100
        for card in self.cards:
            cPrior = card.priority
            
            if card.value == 'J':
                cPrior += 7
            
            if cPrior < minPrior:
                minPrior = cPrior
                lpCard = card
        
        return lpCard  # в месте где эта функция вызывается учесть возможность выбора вальта
    
    # выбор наилучшей комбинации
    def findBestCombination(self, tableCards):
        # для бота нужно реализовать проверку всех возможных сочетаний одной из карт с руки с картами на столе
        # желательно придумать как бы это хоть как-то оптимизировать
        
        # решение в лоб
        # формируем список всех возможных комбинаций карт со стола (в том числе пустой список возможен на случай вальта)
        subLists = [ [] ]
        for a in tableCards:
            subLists += [ s + [a] for s in subLists]
        
        # проверяем все комбинации в сочетании к каждой картой на руке
        bestComb = []
        bestPrior = -3   # начальное значение этой переменной влияет на использование вальта
        for hCard in self.cards:  # карты на руках бота
            for tCards in subLists: 
                if self.checkCombination(tCards, hCard): # проверяем возможна ли такая комбинация
                    sumPrior = hCard.priority + sum([card.priority for card in (tCards)])
                    if sumPrior > bestPrior:
                        bestComb = tCards + [hCard]
                        bestPrior = sumPrior
        
        return bestComb  # если возможных комбинаций нет, вернет пустой список
    
# стол
class Table(object):
    def __init__(self, first_cards):
        self.cardsOnTable = first_cards[:]  # возможно стоит перенести это в addCard (с изменением её на случ неск карт)
    
    # выкладывание карты на стол
    def addCard(self, card):
        self.cardsOnTable.append(card)
    
    # удаление собранных игроком карт со стола
    def removeCards(self, cards):
        # возможно можно как-то улучшить, может с помощью пакета collections
        for card in cards:
            for i, tc in enumerate(self.cardsOnTable):
                if tc == card:
                    self.cardsOnTable.pop(i)
                    break


#class Play(object):
#    def __init__(self, players_num):
#        self.players = [Player() for i in players_num]
#        self.lastTaker = ''
#        self.deck = Deck11()
#        self.deck.shuffle()